In [549]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss

In [394]:
train_x = pd.read_csv("C:/Users/kxj133/Desktop/AV2/train.csv")
test = pd.read_csv("C:/Users/kxj133/Desktop/AV2/test.csv")

train_x.drop(['redemption_status'],axis =1,inplace = True)

train_x['train'] = "Yes"
test['train'] = "No"

train = pd.concat([train_x,test],axis =0,sort = False).reset_index()
train.drop('index',inplace = True,axis =1)

In [395]:
train.tail()

,id,campaign_id,coupon_id,customer_id,train
128590,128584,20,843,501,No
128591,128588,25,415,481,No
128592,128591,25,596,1336,No
128593,128593,22,518,748,No
128594,128594,18,988,851,No


In [399]:
campaign_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/campaign_data.csv")
coupon_item_mapping = pd.read_csv("C:/Users/kxj133/Desktop/AV2/coupon_item_mapping.csv")
customer_demographics = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_demographics.csv")
customer_transaction_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_transaction_data.csv")
item_data =  pd.read_csv("C:/Users/kxj133/Desktop/AV2/item_data.csv")

In [342]:
item_data.head(5)

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [369]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,actual_price,diff_actual_selling_price
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,45.95,10.69
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,67.32,13.89
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,120.75,14.25
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,67.32,0.00
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,99.38,28.14


In [401]:
customer_transaction_data['actual_price'] = customer_transaction_data["selling_price"] + (customer_transaction_data['other_discount'] * (-1)) + (customer_transaction_data['coupon_discount'] * (-1)) 
customer_transaction_data['diff_actual_selling_price'] = customer_transaction_data['actual_price']- customer_transaction_data['selling_price']
customer_transaction_data['ratio_actual_selling_price'] = customer_transaction_data['selling_price']/customer_transaction_data['actual_price'] 
###customer_transaction_data aggregation
#customer_transaction_data.groupby('customer_id')['selling_price'].sum()

max_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','selling_price'].max()
max_purchase = max_purchase.rename(columns = {'item_id' : 'max_sell_price_item',"selling_price" : 'max_sell_price'})
max_actual_price_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','actual_price'].max()
max_actual_price_purchase = max_actual_price_purchase.rename(columns = {'item_id' : 'max_actual_price_item',"actual_price" : 'max_actual_price'})

max_actual_max_sell = pd.concat([max_actual_price_purchase,max_purchase],axis =1)

max_actual_max_sell = max_actual_max_sell.reset_index()
max_actual_max_sell.drop(['max_actual_price_item'],axis =1,inplace = True)

max_actual_max_sell['max_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].max())
max_actual_max_sell['min_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].min())
max_actual_max_sell['max_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].max())
max_actual_max_sell['min_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].min())

max_actual_max_sell['diff_max_actual_max_sell_price'] = max_actual_max_sell['max_actual_price'] - max_actual_max_sell['max_sell_price']
max_actual_max_sell['ratio_max_actual_max_sell_price'] = max_actual_max_sell['max_sell_price']/max_actual_max_sell['max_actual_price']

max_actual_max_sell['purchase_count'] = max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].count())
max_actual_max_sell['coupon_usage_count'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].count()).fillna(0)
max_actual_max_sell['ratio_purchase_count_discount_count'] = max_actual_max_sell['coupon_usage_count']/max_actual_max_sell['purchase_count']
max_actual_max_sell['coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].sum()).fillna(0)
max_actual_max_sell['max_coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].min()).fillna(0)

####in which transaction he used his first coupon
coupon_first_day = customer_transaction_data[["customer_id","date","coupon_discount"]].groupby(['customer_id','date'],sort = True).sum().reset_index().groupby(['customer_id']).head(1).reset_index()
coupon_first_day.drop(['index','date'],axis =1,inplace = True)

max_actual_max_sell = pd.merge(max_actual_max_sell,coupon_first_day,how = "left",on = "customer_id")

### Since the min_diff_actual_sell_price and max_ratio_actual_selling_price is same for all the customers we are removing that that variable
max_actual_max_sell.drop(['min_diff_actual_sell_price','max_ratio_actual_selling_price'],axis =1,inplace = True)

In [402]:
coupon_item_mapping['item_list'] =  coupon_item_mapping['coupon_id'].map(coupon_item_mapping.groupby(["coupon_id"])["item_id"].apply(list))

coupon_item_mapping.drop(['item_id'],axis =1,inplace = True)

coupon_item_mapping.drop_duplicates(subset = ['coupon_id'],inplace= True)

In [442]:
merged_data = pd.merge(train,campaign_data,how = "left",on = "campaign_id" )

merged_data = pd.merge(merged_data,coupon_item_mapping,how = "left",on = "coupon_id")

merged_data = pd.merge(merged_data,customer_demographics,how = "left" , on = "customer_id")

In [443]:
## transaction data
merged_data['total_purchase_price'] = merged_data['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].sum())
merged_data = pd.merge(merged_data,max_purchase,how = "left", on = "customer_id")

In [444]:
merged_data['len_item_list'] = merged_data['item_list'].apply(lambda x : len(x))

merged_data.drop('item_list',axis =1,inplace = True)

In [445]:
merged_data[merged_data['customer_id'] == 1].head(2)

,id,campaign_id,coupon_id,customer_id,train,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,total_purchase_price,max_sell_price_item,max_sell_price,len_item_list
2815,4674,29,597,1,Yes,Y,08/10/12,30/11/12,70+,Married,0.0,2,NaN,4.0,98513.93,73348,445.25,63
3062,5085,8,424,1,Yes,X,16/02/13,05/04/13,70+,Married,0.0,2,NaN,4.0,98513.93,73348,445.25,3


In [446]:
merged_data.drop(['age_range','marital_status','rented','family_size','no_of_children','income_bracket'],axis =1,inplace = True)

In [447]:
merged_data = pd.merge(merged_data,max_actual_max_sell,how = "left", on = "customer_id",sort = True)

In [458]:
merged_data['start_date'] = pd.to_datetime(merged_data['start_date'],format = "%d/%m/%y")

merged_data['end_date'] = pd.to_datetime(merged_data['end_date'],format = "%d/%m/%y")

In [485]:
merged_data['campaign_days'] = merged_data['end_date'] - merged_data['start_date']

In [467]:
merged_data.head()

,id,campaign_id,coupon_id,customer_id,train,campaign_type,start_date,end_date,total_purchase_price,max_sell_price_item_x,...,min_ratio_actual_selling_price,diff_max_actual_max_sell_price,ratio_max_actual_max_sell_price,purchase_count,coupon_usage_count,ratio_purchase_count_discount_count,coupon_usage_amt,max_coupon_usage_amt,coupon_discount,campaign_days
0,4674,29,597,1,Yes,Y,2012-10-08,2012-11-30,98513.93,73348,...,0.358163,461.28,0.491159,1048,78.0,0.074427,-2116.83,-106.86,-35.62,53 days
1,5085,8,424,1,Yes,X,2013-02-16,2013-04-05,98513.93,73348,...,0.358163,461.28,0.491159,1048,78.0,0.074427,-2116.83,-106.86,-35.62,48 days
2,5380,8,103,1,Yes,X,2013-02-16,2013-04-05,98513.93,73348,...,0.358163,461.28,0.491159,1048,78.0,0.074427,-2116.83,-106.86,-35.62,48 days
3,6741,29,896,1,Yes,Y,2012-10-08,2012-11-30,98513.93,73348,...,0.358163,461.28,0.491159,1048,78.0,0.074427,-2116.83,-106.86,-35.62,53 days
4,9849,12,705,1,Yes,Y,2013-04-22,2013-05-24,98513.93,73348,...,0.358163,461.28,0.491159,1048,78.0,0.074427,-2116.83,-106.86,-35.62,32 days


In [490]:
dir(merged_data['start_date'].dt)

['__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_get_values',
 '_reset_cache',
 'ceil',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'microsecond',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'normalize',
 'quarter',
 'round',
 'sec

In [499]:
merged_data['start_date'].dt.quarter

0         4
1         1
2         1
3         4
4         2
5         1
6         4
7         1
8         2
9         2
10        4
11        1
12        2
13        4
14        2
15        4
16        4
17        2
18        4
19        2
20        1
21        2
22        4
23        4
24        4
25        1
26        1
27        1
28        2
29        4
         ..
128565    3
128566    3
128567    3
128568    3
128569    3
128570    3
128571    3
128572    3
128573    3
128574    3
128575    3
128576    3
128577    3
128578    3
128579    3
128580    3
128581    3
128582    3
128583    3
128584    3
128585    3
128586    3
128587    3
128588    3
128589    3
128590    3
128591    3
128592    3
128593    3
128594    3
Name: start_date, Length: 128595, dtype: int64

In [501]:
merged_data['start_date_month_end'] = merged_data['start_date'].dt.is_month_end
merged_data['end_date_month_end'] = merged_data['end_date'].dt.is_month_end
merged_data['start_date_year_end'] = merged_data['start_date'].dt.is_year_end
merged_data['start_date_quarter'] = merged_data['start_date'].dt.quarter

In [506]:
drop_columns = ['campaign_id','coupon_id','start_date','end_date']
merged_data.drop(drop_columns, axis = 1,inplace = True)

In [508]:
merged_data.head(2)

,id,customer_id,train,campaign_type,total_purchase_price,max_sell_price_item_x,max_sell_price_x,len_item_list,max_actual_price,max_sell_price_item_y,...,coupon_usage_count,ratio_purchase_count_discount_count,coupon_usage_amt,max_coupon_usage_amt,coupon_discount,campaign_days,start_date_month_end,end_date_month_end,start_date_year_end,start_date_quarter
0,4674,1,Yes,Y,98513.93,73348,445.25,63,906.53,73348,...,78.0,0.074427,-2116.83,-106.86,-35.62,53 days,False,True,False,4
1,5085,1,Yes,X,98513.93,73348,445.25,3,906.53,73348,...,78.0,0.074427,-2116.83,-106.86,-35.62,48 days,False,False,False,1


In [519]:
merged_data['customer_id'].nunique()/train_data.shape[0]

0.0201865533565568

In [525]:
train_data = merged_data.loc[merged_data['train'] == 'Yes']
test_data = merged_data.loc[merged_data['train'] == 'No']

In [523]:
data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/train.csv")

In [524]:
data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status'], dtype='object')

In [543]:
final_data = pd.merge(train_data,data[['id','redemption_status']], how = 'left',on = 'id')

In [544]:
final_data.head(2)

,id,customer_id,train,campaign_type,total_purchase_price,max_sell_price_item_x,max_sell_price_x,len_item_list,max_actual_price,max_sell_price_item_y,...,ratio_purchase_count_discount_count,coupon_usage_amt,max_coupon_usage_amt,coupon_discount,campaign_days,start_date_month_end,end_date_month_end,start_date_year_end,start_date_quarter,redemption_status
0,4674,1,Yes,Y,98513.93,73348,445.25,63,906.53,73348,...,0.074427,-2116.83,-106.86,-35.62,53 days,False,True,False,4,0
1,5085,1,Yes,X,98513.93,73348,445.25,3,906.53,73348,...,0.074427,-2116.83,-106.86,-35.62,48 days,False,False,False,1,0


In [545]:
final_data.drop(['train','customer_id','id','campaign_days'],axis =1,inplace = True)

In [546]:
le = LabelEncoder()
final_data['campaign_type'] = le.fit_transform(final_data['campaign_type'])

In [547]:
X = final_data.drop('redemption_status',axis =1)
Y = final_data['redemption_status']
### test train split
train_x,test_x,train_y,test_y = train_test_split(X,Y,random_state = 1000,test_size = 0.30)

In [553]:
lgb = LGBMClassifier(nthread=8,
                     n_estimators=400,
                     num_leaves=45,
                     learning_rate=0.03,
                     subsample=1.0,
                     subsample_freq=1,
                     colsample_bytree=0.3,
                     min_child_samples=10)

lgb.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
               importance_type='split', learning_rate=0.03, max_depth=-1,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, nthread=8, num_leaves=45,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=1)

In [562]:
test_data.drop(['train','customer_id','id','campaign_days'],axis =1,inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [565]:
le = LabelEncoder()
test_data['campaign_type'] = le.fit_transform(test_data['campaign_type'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [566]:
predict = lgb.predict(test_data)

In [567]:
submission = pd.read_csv("C:/Users/kxj133/Desktop/AV2/sample_submission.csv")

In [568]:
submission.head()

,id,redemption_status
0,3,0
1,4,0
2,5,0
3,8,0
4,10,0


In [559]:
from sklearn.metrics import accuracy_score,roc_auc_score

In [560]:
roc_auc_score(test_y,predict)

0.5402829081819164